# Bayesian Belief Networks

![Question 1](https://i.imgur.com/3ayVbFi.jpeg) 

All nodes are binary and can take 0/1 values

The probabilities are given below:

P(Bark = 1) = 0.05   
P(Rain = 1) = 0.01


P(Scared = 1 | Bark = 0, Rain = 0) = 0.001  
P(Scared = 1 | Bark = 0, Rain = 1) = 0.1  
P(Scared = 1 | Bark = 1, Rain = 0) = 0.8  
P(Scared = 1 | Bark = 1, Rain = 1) = 0.9  

P(Hides = 1 | Scared = 1) = 0.95  
P(Hides = 1 | Scared = 0) = 0.05  

For the given Bayesian network, Compute the following probabilities :  

#### (a) Find the probability that cat hides = 1.  

P(Scared = 1) = P(B,R) * P(S|B,R) + P(B,R') * P(S|B,R') + P(B',R) * P(S|B',R) + P(B',R') * P(S|B',R')

P(S) = (0.0005)(0.9) + (0.0495)(0.8) + (0.0095)(0.1) + (0.9405)(0.001) = 0.0419405

P(S') = 1 - 0.0419405 = 0.9580595

P(Hides = 1) = P(H|S) * P(S) + P(H|S') * P(S') = (0.95)(0.0419405) + (0.05)(0.9580595) = **0.08774645** = 0.088

#### (b) Given that cat got scared (Scared = 1), what is the probability that it rained (Rain = 1)?  

P(Scared = 1 | Rain = 1) = P(B and R) * P(S|B,R) + P(B' and R) * P(S|B',R) 
  = (0.9)(0.05)(0.01) + (0.1)(0.95)(0.01) = 0.00045 + 0.00095 = 0.0014

P(Rain = 1 | Scared = 1) = P(S|R) * (P(R) / P(S)) = (0.0014) * ((0.01) / 0.0419405) = **0.000333806** = 0.0003

#### (c) Given that cat got scared (Scared = 1) and the dog barked (Bark = 1), what is the probability that it rained (Rain = 1)?

P(Bark = 1, Rain = 1 ,Scared = 1) = P(S|B,R) * P(B) * P(R) = (0.9)(0.05)(0.01) = 0.00045

P(Scared = 1 | Bark = 1) = P(S|B,R) * P(B) * P(R) + P(S|B,R') * P(B) * P(R') = (0.9)(0.05)(0.01) + (0.8)(0.05)(0.99) = 0.00045 + 0.0396 = 0.04005

P(S,B) = P(S|B) * P(B) = (0.04005)(0.05) = 0.0020025

P(Rain = 1 | Scared = 1, Bark = 1) = P(B,R,S) / P(S,B) = 0.00045 / 0.0020025 = **0.224719101** = 0.225

# Comparison of different ensembles method for classification

In this question, we will compare performance of different ensemble methods for classification problems: [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html), [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html), [GradientBoosting](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html), and [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) Classifiers.

[Spam Classification Data](https://archive.ics.uci.edu/ml/datasets/Spambase) of UCI will be used (you can use the data provided: 'spam_uci.csv'). Don't worry about column names. The last column represents target label, 1 if spam and zero otherwise.

* Load the data and partition it into features (X) and the target label (y) for classification task. Then, use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. 

#### Loading and pre-processing dataset

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

spam = pd.read_csv('spam_uci.csv')
spam.drop('Unnamed: 0', axis=1, inplace=True)

X = spam.drop('57', 1)
y = spam['57']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Part 1 Use a Decision Tree Classifier with random_state = 10 and Logistic Regression with random_state = 10 and solver ="lbfgs" for the spam classification problem. Report the accuracy_score and roc_auc_score on the test data for each classifier.

In [2]:
#Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state=10).fit(X_train, y_train)
print('Tree Test Accuracy:', dtc.score(X_test, y_test))

y_score = dtc.predict_proba(X_test)
y_score = y_score[:,1]
print('Tree Test AUC:', roc_auc_score(y_test, y_score))
print()

#Logistic Regression
logr = LogisticRegression(random_state=10,solver='lbfgs',max_iter=6000).fit(X_train, y_train)
print('Logistic Test Accuracy:', logr.score(X_test, y_test))

y_score = logr.predict_proba(X_test)
y_score = y_score[:,1]
print('Logistic Test AUC:', roc_auc_score(y_test, y_score))

Tree Test Accuracy: 0.9117840684660962
Tree Test AUC: 0.9085903594264297

Logistic Test Accuracy: 0.9315339038841343
Logistic Test AUC: 0.9727300931819884


Part 2 Create an ensemble of 50 classifiers (i.e n_estimators = 50) with random_state = 10 for [bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) with base classifier as Decision Tree Classifier and Logistic Regression from part $1$ . Report accuracy_score and roc_auc_score on the test data for both the Bagging classifiers. Compare the results and breifly explain the effect of using bagging for the spam classification problem for both the Logistic Regression and Decision Tree base classifier.

In [3]:
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier(base_estimator=dtc, n_estimators=50,random_state=10).fit(X_train, y_train)
print('Bag + Tree Test Accuracy:', bag.score(X_test, y_test))

y_score = bag.predict_proba(X_test)
y_score = y_score[:,1]
print('Bag + Tree Test AUC:', roc_auc_score(y_test, y_score))
print()

bag = BaggingClassifier(base_estimator=logr, n_estimators=50,random_state=10).fit(X_train, y_train)
print('Bag + Logistic Test Accuracy:', bag.score(X_test, y_test))

y_score = bag.predict_proba(X_test)
y_score = y_score[:,1]
print('Bag + Logistic Test AUC:', roc_auc_score(y_test, y_score))

Bag + Tree Test Accuracy: 0.9447004608294931
Bag + Tree Test AUC: 0.9813261583558889

Bag + Logistic Test Accuracy: 0.9335088874259381
Bag + Logistic Test AUC: 0.9751550358570568


Before bagging, the Logistic Regression model provided better test accuracy than Decision Trees. Although bagging provided almost no improvement when using the Logistic Regression estimator, it significantly improved the accuracy of Decision Tree model. Bagging performs best with algorithms or base estimators that have high variance. A popular example are decision trees, often constructed without pruning (max_depth=None). We could expect a classifier to be more effective than a regression when dealing with classification or discrete classes such as this. 

Part 3 Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to classify whether an email is spam. Report your testing accuracy ([accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score). You will need [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba) for roc_auc_score. 

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[50,100,250], 'max_depth':[5,10,25,None]}

gsc = GridSearchCV(RandomForestClassifier(random_state=10), params)
gsc.fit(X_train, y_train)
print('Best Random Forest Parameters:', gsc.best_params_)

rfc = RandomForestClassifier(random_state=10, max_depth=gsc.best_params_['max_depth'], 
                             n_estimators=gsc.best_params_['n_estimators']).fit(X_train, y_train)

print('Random Forest Test Accuracy:', rfc.score(X_test, y_test))

y_score = rfc.predict_proba(X_test)
y_score = y_score[:,1]
print('Random Forest Test AUC:', roc_auc_score(y_test, y_score))

Best Random Forest Parameters: {'max_depth': 25, 'n_estimators': 100}
Random Forest Test Accuracy: 0.9598420013166556
Random Forest Test AUC: 0.9881231300304187


Part 4 Use [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) for the spam classification problem.  Report  accuracy_score and roc_auc_score on the test data for each algorithm. 

  Note : For part 3 and 4 find the best values for the hyper parameters of each of the models by using GridSearchCV. 

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

params = {'n_estimators':[50,100,250], 'max_depth':[5,10,25,None]}

gsc = GridSearchCV(GradientBoostingClassifier(random_state=10), params)
gsc.fit(X_train, y_train)
print('Best Gradient Parameters:', gsc.best_params_)

gbc = GradientBoostingClassifier(random_state=10,max_depth=gsc.best_params_['max_depth'], 
                             n_estimators=gsc.best_params_['n_estimators']).fit(X_train, y_train)
print('Gradient Test Accuracy:', gbc.score(X_test, y_test))

y_score = gbc.predict_proba(X_test)
y_score = y_score[:,1]
print('Gradient Test AUC:', roc_auc_score(y_test, y_score))
print()

params = {'n_estimators':[50,100,250], 'learning_rate':[.01,.1,1]}

gsc = GridSearchCV(AdaBoostClassifier(random_state=10), params)
gsc.fit(X_train, y_train)
print('Best AdaBoost Parameters:', gsc.best_params_)

ada = AdaBoostClassifier(random_state=10,n_estimators=gsc.best_params_['n_estimators'],
                        learning_rate=gsc.best_params_['learning_rate']).fit(X_train, y_train)
print('AdaBoost Test Accuracy:', ada.score(X_test, y_test))

y_score = ada.predict_proba(X_test)
y_score = y_score[:,1]
print('AdaBoost Test AUC:', roc_auc_score(y_test, y_score))

Best Gradient Parameters: {'max_depth': 5, 'n_estimators': 250}
Gradient Test Accuracy: 0.9572086899275839
Gradient Test AUC: 0.9897350036909054

Best AdaBoost Parameters: {'learning_rate': 0.1, 'n_estimators': 250}
AdaBoost Test Accuracy: 0.9526003949967083
AdaBoost Test AUC: 0.9873163016771332


Part 5 Briefly explain which of the three ensemble method above from Part 3&4 performed the best and why?

The Random Forest ensemble method performed the best when compared to Gradient Boosting and AdaBoost. Random Forests are highly accurate classifiers that run efficiently on large datasets, handling a large number of variables without deletion. Random Forests are also less susceptible to overfitting than gradient boosting methods. Even when tuning the parameters of the Gradient and Adaptive Boosting methods, the Random Forest outperformed.

In gradient boosting, decision trees are built one at a time and are used successively as weak learners. Adaptive Boosting (AdaBoost) generally works by weighting instances in the dataset by how easy or difficult they are to classify, allowing the algorithm to pay or or less attention to them in the construction of subsequent models.